In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Multi-Agent Architecture for Goods Management

#Agents 

class CustomerAgent:
    def place_order(self, sku, qty):
        print(f"[CustomerAgent] Order placed for {qty} units of {sku}")
        return {"sku": sku, "qty": qty}
class InventoryAgent:
    def __init__(self):
        self.stock = {"SKU-1": 10, "SKU-2": 3, "SKU-3": 7}
        self.threshold = {"SKU-1": 5, "SKU-2": 4, "SKU-3": 5}
    def check_and_allocate(self, order):
        sku = order["sku"]
        qty = order["qty"]
        if sku not in self.stock:
            print("[InventoryAgent] Product does not exist.")
            return "cancelled"
        if self.stock[sku] >= qty:
            print("[InventoryAgent] Sufficient stock. Allocating...")
            self.stock[sku] -= qty
            return "allocated"
        print("[InventoryAgent] Low stock! Triggering Supplier Agent...")
        return "low_stock"
    def update_stock(self, sku, qty):
        self.stock[sku] += qty
        print(f"[InventoryAgent] Stock updated for {sku}. New qty = {self.stock[sku]}")
class SupplierAgent:
    def supply_goods(self, sku, qty, inventory_agent):
        print(f"[SupplierAgent] Supplying {qty} units of {sku}")
        inventory_agent.update_stock(sku, qty)
class DeliveryAgent:
    def __init__(self):
        # Simple road-map graph
        self.graph = {
            "warehouse": {"cityA": 5, "hub1": 3},
            "hub1": {"cityA": 2, "cityB": 6},
            "cityA": {},
            "cityB": {}
        }
    def shortest_path(self, start, end):
        unvisited = {node: float('inf') for node in self.graph}
        unvisited[start] = 0
        visited = {}
        parents = {}
        while unvisited:
            curr_node = min(unvisited, key=unvisited.get)
            curr_cost = unvisited[curr_node]
            if curr_node == end:
                break
            for neighbor, cost in self.graph[curr_node].items():
                new_cost = curr_cost + cost
                if new_cost < unvisited.get(neighbor, float('inf')):
                    unvisited[neighbor] = new_cost
                    parents[neighbor] = curr_node
            visited[curr_node] = curr_cost
            unvisited.pop(curr_node)
        # Reconstruct path
        path = []
        node = end
        while node in parents:
            path.insert(0, node)
            node = parents[node]
        path.insert(0, start)

        return path, visited.get(end, float('inf'))

    def deliver(self, order):
        dest = "cityA"
        path, cost = self.shortest_path("warehouse", dest)
        print(f"[DeliveryAgent] Delivering {order['qty']} units of {order['sku']}")
        print(f"[DeliveryAgent] Route: {' -> '.join(path)} (Cost: {cost})")
        print("[DeliveryAgent] Delivery completed!")

# Main Workflow

customer = CustomerAgent()
inventory = InventoryAgent()
supplier = SupplierAgent()
delivery = DeliveryAgent()

# Step 1: Customer places an order
order = customer.place_order("SKU-2", 5)

# Step 2: Inventory tries to allocate
result = inventory.check_and_allocate(order)

# Step 3: If low stock → Supplier supplies
if result == "low_stock":
    supplier.supply_goods("SKU-2", 10, inventory)
    print("[System] Retrying order allocation...")
    result = inventory.check_and_allocate(order)

# Step 4: If allocated → Deliver
if result == "allocated":
    delivery.deliver(order)

print("\nFinal Inventory:", inventory.stock)

[CustomerAgent] Order placed for 5 units of SKU-2
[InventoryAgent] Low stock! Triggering Supplier Agent...
[SupplierAgent] Supplying 10 units of SKU-2
[InventoryAgent] Stock updated for SKU-2. New qty = 13
[System] Retrying order allocation...
[InventoryAgent] Sufficient stock. Allocating...
[DeliveryAgent] Delivering 5 units of SKU-2
[DeliveryAgent] Route: warehouse -> cityA (Cost: inf)
[DeliveryAgent] Delivery completed!

Final Inventory: {'SKU-1': 10, 'SKU-2': 8, 'SKU-3': 7}
